# Mujeres, anemia, vivienda, hogar

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from unicodedata import normalize
import copy
import json

# Load datasets to merge

In [2]:
general_file_path = '../files/01_csv'
csv_prefix = 'AnyConv.com__'

In [3]:
rech0_df = pd.read_csv(f'{general_file_path}/{csv_prefix}RECH0.csv') # HOGAR
rech23_df = pd.read_csv(f'{general_file_path}/{csv_prefix}RECH23.csv') # VIVIENDA
rech5_df = pd.read_csv(f'{general_file_path}/{csv_prefix}RECH5.csv') # MUJERES - ANEMIA

In [6]:
rech0_df.columns = [column.split(',')[0] for column in rech0_df.columns]
rech23_df.columns = [column.split(',')[0] for column in rech23_df.columns]
rech5_df.columns = [column.split(',')[0] for column in rech5_df.columns]

In [7]:
print(rech0_df.shape)
print(rech23_df.shape)
print(rech5_df.shape)

(37474, 44)
(37474, 131)
(39989, 38)


In [8]:
# drop HV237H HV237H HV237I HV237I HV237J HV237J HV237K HV237K (duplicado)
rech23_df = rech23_df.drop(['HV237H', 'HV237H', 'HV237I', 'HV237I', 'HV237J', 'HV237J', 'HV237K', 'HV237K'], axis=1)

In [9]:
rech5_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39989 entries, 0 to 39988
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID1     39989 non-null  int64  
 1   HHID    39989 non-null  int64  
 2   HA0     39989 non-null  int64  
 3   HA1     39989 non-null  int64  
 4   HA2     39989 non-null  int64  
 5   HA3     39989 non-null  int64  
 6   HA4     39989 non-null  int64  
 7   HA5     39989 non-null  int64  
 8   HA6     39989 non-null  int64  
 9   HA11    39989 non-null  int64  
 10  HA12    39989 non-null  int64  
 11  HA12A   39989 non-null  int64  
 12  HA12B   39989 non-null  int64  
 13  HA13    39989 non-null  int64  
 14  HA32    39989 non-null  int64  
 15  HA33    39989 non-null  int64  
 16  HA35    39989 non-null  int64  
 17  HA40    39989 non-null  int64  
 18  HA41    39989 non-null  int64  
 19  HA50    39989 non-null  int64  
 20  HA51    39989 non-null  int64  
 21  HA52    39989 non-null  int64  
 22

# Unique ID

In [12]:
rech5_df['HHID'] = rech5_df['HHID'].astype(str)
rech5_df['HA0'] = rech5_df['HA0'].astype(str)

In [13]:
rech5_df['UID'] = rech5_df['HHID'] + '-' + rech5_df['HA0']

In [14]:
# to str hhid
rech0_df['HHID'] = rech0_df['HHID'].astype(str)
rech23_df['HHID'] = rech23_df['HHID'].astype(str)

rech0_df.drop('ID1', inplace=True, axis=1)
rech23_df.drop('ID1', inplace=True, axis=1)

In [15]:
full_df = rech5_df.merge(rech0_df, on = 'HHID')
full_df = full_df.merge(rech23_df, on = 'HHID')

In [16]:
full_df.shape

(39989, 206)

In [17]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39989 entries, 0 to 39988
Columns: 206 entries, ID1 to HV271
dtypes: float64(3), int64(198), object(5)
memory usage: 63.2+ MB


In [18]:
# Raw joined dataset
datasets_path = '../files/03_datasets'

# RAW join to disk

In [19]:
full_df.to_csv(f'{datasets_path}/raw_mujeres_anemia_hv.csv')

# Creating labels and columns

In [22]:
csv_path = '../files/02_dictionaries/01_csv_load'
json_path = '../files/02_dictionaries/02_json_mapping'

In [29]:
def transform_spanish(s):
    s = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", s), 0, re.I
    )

    # -> NFC
    s = normalize( 'NFC', s)
    s = s.replace(u"ñ","nh")
    s = s.lower()
    s = s.replace(" ","_")
    return s

In [30]:
info_rech5_df = pd.read_csv(f'{csv_path}/RECH5.csv', encoding = "ISO-8859-1", engine='python') # MUJERES - ANEMIA

In [32]:
# Transform descriptions
info_rech5_df["DESC_CLEAN"] = info_rech5_df["DESCRIPCIÓN DE LAS VARIABLES"].apply(lambda x: transform_spanish(x))
# Group mappings
group_info_rech5 = info_rech5_df.groupby(by=["VARIABLE"])

# Create dict of mappings
dict_mapping = {}
for index, df in group_info_rech5:
    if df['VALORES'].isnull().all() == False:
        print(index)
        #print(df['DESC_CLEAN'])
        new_key = df['DESC_CLEAN'].values[0]
        print(new_key)
        dict_valor = {}
        for sub_index, valor, desc in df[['VALORES','DESCRIPCIÓN DE LAS ALTERNATIVAS']].itertuples():
            dict_valor[int(valor)] = desc      
        dict_mapping[new_key] = copy.deepcopy(dict_valor)

# idcolumns and decs columns
columns_mapping_df = info_rech5_df[['VARIABLE','DESC_CLEAN']].drop_duplicates(keep = 'first')

# For the columns
dict_columnas = {}
for sub_index, valor, desc in columns_mapping_df.itertuples():
    dict_columnas[valor] = desc  
    
# Save
with open(f'{json_path}/RECH5/labels.json', 'w') as fp:
    json.dump(dict_mapping, fp)

print("Labels saved")
    
with open(f'{json_path}/RECH5/columns.json', 'w') as fp:
    json.dump(dict_columnas, fp)
    
print("Columns saved")

HA11
peso/talla_desvio_estandar_(dhs)
HA12
peso/talla_porcentaje_respecto_a_la_mediana_(dhs)
HA12A
peso/talla_porcentaje_respecto_a_la_mediana_(foggarty)
HA12B
peso/talla_porcentaje_respecto_a_la_mediana_(oms)
HA13
resultado_de_medicion_de_la_entrevistada
HA33
fecha_de_informacion_completa
HA35
en_las_ultimas_24_horas_cuantos_cigarrillos_fumo_ud.
HA4
talla/edad_percentil_en_que_se_encuentra
HA40
indice_de_masa_corporal_para_la_entrevistada
HA41
indice_de_rohrer_para_la_entrevistada
HA5
talla/edad_desviacion_estandar
HA50
grupos_de_mujeres_de_12_a_49_anhos_de_edad
HA51
numero_de_orden_del_padre_o_responsable
HA52
leyo_la_declaracion_de_consentimiento_para_la_medicion_de_hemoglobina
HA54
actualmente_embarazada
HA55
resultado_de_la_medicion_(hemoglobina)
HA57
nivel_de_anemia
HA58
esta_de_acuerdo_con_enviar_los_datos_a_un_especialista
HA6
talla/edad_porcentaje_respecto_a_la_mediana_de_referencia
HA60
estado_civil
HA61
lea_la_declaracion_de_consentimiento_(vih)
HA63
resultado_de_la_medicion

# Create filled dataframe

In [40]:
with open(f'{json_path}/RECH0/columns.json') as json_file:
    columns_rech0_dict = json.load(json_file)

with open(f'{json_path}/RECH23/columns.json') as json_file:
    columns_rech23_dict = json.load(json_file)

with open(f'{json_path}/RECH5/columns.json') as json_file:
    columns_rech5_dict = json.load(json_file)
    

In [41]:
with open(f'{json_path}/RECH0/labels.json') as json_file:
    labels_rech0_dict = json.load(json_file)
with open(f'{json_path}/RECH23/labels.json') as json_file:
    labels_rech23_dict = json.load(json_file)
with open(f'{json_path}/RECH5/labels.json') as json_file:
    labels_rech5_dict = json.load(json_file)
    

In [36]:
# Loading dataset
original_raw_joined_df = pd.read_csv(f'{datasets_path}/raw_mujeres_anemia_hv.csv')
original_raw_joined_df = original_raw_joined_df.drop(original_raw_joined_df.columns[0], axis=1) #olvide quitarle el index lol

In [37]:
original_raw_joined_df.head()

,ID1,HHID,HA0,HA1,HA2,HA3,HA4,HA5,HA6,HA11,...,SH78,SH79,SH224,SH225U,SH225,SH227,QH227A,QH227B,HV270,HV271
0,2019,100201,2,33,593,1500,108,-230,9163,37,...,1,20,4,2,16,1,1,1,4,1
1,2019,100201,3,12,504,1508,3128,-49,9786,-81,...,1,20,4,2,16,1,1,1,4,1
2,2019,102801,2,36,843,1605,2953,-54,9804,174,...,0,0,4,1,37,1,2,1,4,1
3,2019,102801,6,49,673,1618,3745,-32,9883,24,...,0,0,4,1,37,1,2,1,4,1
4,2019,104801,2,28,851,1579,1651,-97,9645,246,...,0,0,3,2,16,2,1,1,2,0


In [38]:
original_raw_joined_df.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39989 entries, 0 to 39988
Data columns (total 206 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ID1        int64  
 1   HHID       int64  
 2   HA0        int64  
 3   HA1        int64  
 4   HA2        int64  
 5   HA3        int64  
 6   HA4        int64  
 7   HA5        int64  
 8   HA6        int64  
 9   HA11       int64  
 10  HA12       int64  
 11  HA12A      int64  
 12  HA12B      int64  
 13  HA13       int64  
 14  HA32       int64  
 15  HA33       int64  
 16  HA35       int64  
 17  HA40       int64  
 18  HA41       int64  
 19  HA50       int64  
 20  HA51       int64  
 21  HA52       int64  
 22  HA53       int64  
 23  HA54       int64  
 24  HA55       int64  
 25  HA56       int64  
 26  HA57       int64  
 27  HA58       int64  
 28  HA60       int64  
 29  HA61       int64  
 30  HA62       float64
 31  HA63       int64  
 32  HA64       int64  
 33  HA65       int64  
 34  HA66       int64  
 3

## Replace columns

In [42]:
original_raw_joined_df.rename(columns=columns_rech0_dict, 
                 inplace=True)
original_raw_joined_df.rename(columns=columns_rech23_dict, 
                 inplace=True)
original_raw_joined_df.rename(columns=columns_rech5_dict, 
                 inplace=True)

In [43]:
original_raw_joined_df.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39989 entries, 0 to 39988
Data columns (total 206 columns):
 #   Column                                                                             Dtype  
---  ------                                                                             -----  
 0   anho                                                                               int64  
 1   identificacion_cuestionario_del_hogar                                              int64  
 2   numero_de_orden_en_el_hogar                                                        int64  
 3   edad_de_la_mujer_en_anhos                                                          int64  
 4   peso_en_kilogramos_(1_decimal)                                                     int64  
 5   talla_en_centimetros_(1_decimal)                                                   int64  
 6   talla/edad_percentil_en_que_se_encuentra                                           int64  
 7   talla/edad_desviacion

In [44]:
# Mapping and creating new columns
print('\nHOGAR\n\n')
for key, dict_values in labels_rech0_dict.items():
    print(key)
    print(dict_values)
    original_raw_joined_df[f'{key}_LABEL'] = original_raw_joined_df[key].apply(lambda x: dict_values[str(int(x))] if str(int(x)) in dict_values else x)


print('\nVIVIENDA\n\n')
for key, dict_values in labels_rech23_dict.items():
    print(key)
    print(dict_values)
    original_raw_joined_df[f'{key}_LABEL'] = original_raw_joined_df[key].apply(lambda x: dict_values[str(int(x))] if str(int(x)) in dict_values else x)

print('\nANEMIA\n\n')
for key, dict_values in labels_rech5_dict.items():
    print(key)
    print(dict_values)
    original_raw_joined_df[f'{key}_LABEL'] = original_raw_joined_df[key].apply(lambda x: dict_values[str(int(x))] if str(int(x)) in dict_values else x)


HOGAR


resultado_de_la_entrevista_en_el_hogar
{'1': 'Completo', '2': 'Hogar presente pero entrevistado ausente', '3': 'Hogar ausente', '4': 'Aplazada', '5': 'Rechazada', '6': 'Vivienda desocupada o no es vivienda', '7': 'Vivienda destruida', '8': 'Vivienda no encontrada', '9': 'Otro'}
dominio
{'1': 'Amazonas', '2': 'Ancash', '3': 'Apurimac', '4': 'Arequipa', '5': 'Ayacucho', '6': 'Cajamarca', '7': 'Callao', '8': 'Cusco', '9': 'Huancavelica', '10': 'Huanuco', '11': 'Ica', '12': 'Junin', '13': 'La Libertad', '14': 'Lambayeque', '15': 'Lima', '16': 'Loreto', '17': 'Madre de Dios', '18': 'Moquegua', '19': 'Pasco', '20': 'Piura', '21': 'Puno', '22': 'San Martin', '23': 'Tacna', '24': 'Tumbes', '25': 'Ucayali'}
region
{'1': 'Amazonas', '2': 'Ancash', '3': 'Apurimac', '4': 'Arequipa', '5': 'Ayacucho', '6': 'Cajamarca', '7': 'Callao', '8': 'Cusco', '9': 'Huancavelica', '10': 'Huanuco', '11': 'Ica', '12': 'Junin', '13': 'La Libertad', '14': 'Lambayeque', '15': 'Lima', '16': 'Loreto', '17': 'M

In [45]:
original_raw_joined_df.head()

,anho,identificacion_cuestionario_del_hogar,numero_de_orden_en_el_hogar,edad_de_la_mujer_en_anhos,peso_en_kilogramos_(1_decimal),talla_en_centimetros_(1_decimal),talla/edad_percentil_en_que_se_encuentra,talla/edad_desviacion_estandar,talla/edad_porcentaje_respecto_a_la_mediana_de_referencia,peso/talla_desvio_estandar_(dhs),...,esta_de_acuerdo_con_enviar_los_datos_a_un_especialista_LABEL,talla/edad_porcentaje_respecto_a_la_mediana_de_referencia_LABEL,estado_civil_LABEL,lea_la_declaracion_de_consentimiento_(vih)_LABEL,resultado_de_la_medicion_(vih)_LABEL,consentimiento_para_pruebas_adicionales_LABEL,resultado_de_la_entrevista_individual_a_la_mujer_LABEL,el_nivel_educativo_mas_alto_aprobado_de_la_mujer_LABEL,el_anho_mas_alto_de_educacion_de_la_mujer_LABEL,nivel_educativo_mas_alto_(cs_para_el_informe_preliminar_y_final)_LABEL
0,2019,100201,2,33,593,1500,108,-230,9163,37,...,No,9163,0,0,0,0,Completa,Superior,5,Superior
1,2019,100201,3,12,504,1508,3128,-49,9786,-81,...,No,9786,0,0,0,0,Completa,Primaria,6,Primaria
2,2019,102801,2,36,843,1605,2953,-54,9804,174,...,No,9804,0,0,0,0,Completa,Secundaria,5,Secundaria
3,2019,102801,6,49,673,1618,3745,-32,9883,24,...,No,9883,0,0,0,0,Completa,Primaria,4,Primaria
4,2019,104801,2,28,851,1579,1651,-97,9645,246,...,No,9645,0,0,0,0,Completa,Primaria,1,Primaria


In [ ]:
original_raw_joined_df.to_csv(f'{datasets_path}/original_whole_mujeres_anemia_vivienda_hogar.csv', index=None)